In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import snapshot_download
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("Supports BF16:", torch.cuda.is_bf16_supported())

CUDA available: True
GPU name: NVIDIA GeForce RTX 5070 Ti
Supports BF16: True


In [3]:
# # 스냅샷 로컬에 저장
# repo_id = "kakaocorp/kanana-nano-2.1b-instruct"
# local_dir = "./model/kanana-nano-2.1b-instruct"

# snapshot_download(repo_id=repo_id, local_dir=local_dir)

In [4]:
local_dir = "./model/kanana-nano-2.1b-instruct"

base_model = AutoModelForCausalLM.from_pretrained(local_dir)

tokenizer = AutoTokenizer.from_pretrained(local_dir, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# 엑셀 파일에서 데이터 읽기
df = pd.read_excel('./data/emotion_data.xlsx')

# NaN 값을 빈 문자열로 처리
df = df.fillna('')

# 데이터를 리스트 형식으로 변환
data_list = []
for _, row in df.iterrows():
    data_list.append({
        'emotion': row['감정_소분류'],
        '사람문장1': row['사람문장1'],
        '시스템문장1': row['시스템문장1'],
        '사람문장2': row['사람문장2'],
        '시스템문장2': row['시스템문장2'],
        '사람문장3': row['사람문장3'],
        '시스템문장3': row['시스템문장3'],
    })

dataset = Dataset.from_dict({key: [d[key] for d in data_list] for key in data_list[0].keys()})

In [13]:
def formatting_prompts_func(examples):
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}


formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
formatted_dataset = formatted_dataset.remove_columns(['Unnamed: 0'])
formatted_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

KeyError: 'instruction'

In [11]:
def formatting_prompts_func(examples):
    """
    멀티턴 대화를 다음 형식으로 변환:
    사람문장1
    시스템문장1
    사람문장2
    시스템문장2
    사람문장3
    시스템문장3
    """
    
    texts = []
    EOS_TOKEN = tokenizer.eos_token
    
    for i in range(len(examples['사람문장1'])):
        # 각 턴의 데이터 추출
        turns = []
        for turn in range(1, 4):  # 사람문장1~3, 시스템문장1~3
            person = examples[f'사람문장{turn}'][i].strip()
            system = examples[f'시스템문장{turn}'][i].strip()
            
            if person:  # 비어있지 않으면 추가
                turns.append(f"사람: {person}")
            if system:
                turns.append(f"시스템: {system}")
        
        # 대화를 하나의 텍스트로 연결
        if turns:
            text = "\n".join(turns) + EOS_TOKEN
            texts.append(text)
    
    return {"text": texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
formatted_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': '',
 'text': '사람: 일은 왜 해도 해도 끝이 없을까? 화가 난다.\n시스템: 많이 힘드시겠어요. 주위에 의논할 상대가 있나요?\n사람: 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.\n시스템: 혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.<|eot_id|>'}

In [6]:
def formatting_prompts_func(examples):
    """
    멀티턴 대화를 다음 형식으로 변환:
    사람문장1
    시스템문장1
    사람문장2
    시스템문장2
    사람문장3
    시스템문장3
    """
    
    texts = []
    
    for i in range(len(examples['사람문장1'])):
        # 각 턴의 데이터 추출
        messages = []

        for turn in range(1, 4):  # 사람문장1~3, 시스템문장1~3
            user = examples[f'사람문장{turn}'][i].strip()
            assistant = examples[f'시스템문장{turn}'][i].strip()
            
            if user:  # 비어있지 않으면 추가
                messages.append({
                    "role": "user",
                    "content": user.strip()
                })
            if assistant:
                messages.append({
                    "role": "assistant",
                    "content": assistant.strip()
                })
        
        # 대화를 하나의 텍스트로 연결
        if messages:
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
    
    return {"text": texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
formatted_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': '',
 'text': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n일은 왜 해도 해도 끝이 없을까? 화가 난다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n많이 힘드시겠어요. 주위에 의논할 상대가 있나요?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.<|eot_id|>'}

In [7]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding="max_length",  # 최대 길이로 padding
        truncation=True,
        max_length=512,  # 필요시 조정
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

In [8]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
        "q_proj", 
        "k_proj", 
        "v_proj"]
)
model = get_peft_model(base_model, lora_config)

In [11]:
from transformers import DataCollatorWithFlattening

# data_collator = DataCollatorWithFlattening()

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs=1,
        max_steps=50,
        learning_rate = 2e-4, 
        bf16 = True,
        logging_steps = 1,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1234,
        output_dir = "kanana_flattening",
        report_to = "none"
    )
)

In [12]:
# 모델 훈련
trainer_stats = trainer.train()

KeyboardInterrupt: 

In [23]:
def formatting_prompts_func(examples):
    all_messages = []

    for i in range(len(examples["사람문장1"])):
        messages = []

        for turn in range(1, 4):
            user = examples[f"사람문장{turn}"][i].strip()
            assistant = examples[f"시스템문장{turn}"][i].strip()

            if user:
                messages.append({"role": "user", "content": user})
            if assistant:
                messages.append({"role": "assistant", "content": assistant})

        if messages:
            all_messages.append(messages)

    return {"messages": all_messages}


formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
formatted_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': '',
 'messages': [{'content': '일은 왜 해도 해도 끝이 없을까? 화가 난다.', 'role': 'user'},
  {'content': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', 'role': 'assistant'},
  {'content': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', 'role': 'user'},
  {'content': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.',
   'role': 'assistant'}]}

In [29]:
from trl import SFTTrainer, SFTConfig

new_template = """{% for message in messages %}
{% if loop.index0 == 0 %}{{ bos_token }}{% endif %}
<|start_header_id|>{{ message['role'] }}<|end_header_id|>

{% if message['role'] == 'assistant' %}
{% generation %}
{{ message['content'] | trim }}
{% endgeneration %}
{% else %}
{{ message['content'] | trim }}
{% endif %}
<|eot_id|>
{% endfor %}

{% if add_generation_prompt %}
<|start_header_id|>assistant<|end_header_id|>
{% endif %}
"""
tokenizer.chat_template = new_template

sft_args = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=50,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=1,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=1234,
    output_dir="outputs_sft_assistant_only",
    report_to="none",

    # 🔥 핵심 (이건 맞았다)
    assistant_only_loss=True,

    # 🔥 여기 수정
    max_length=2048,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=sft_args,
    train_dataset=formatted_dataset,
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/51630 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/51630 [00:00<?, ? examples/s]

In [30]:
trainer.train()

Step,Training Loss
1,0.951100
2,1.186400
3,1.313600
4,1.059200
5,1.289100
6,1.180000
7,1.152100
8,1.084600
9,1.144800
10,1.278500


TrainOutput(global_step=50, training_loss=1.276916241645813, metrics={'train_runtime': 66.5986, 'train_samples_per_second': 6.006, 'train_steps_per_second': 0.751, 'total_flos': 747964735604736.0, 'train_loss': 1.276916241645813, 'epoch': 0.007747433662599264})